# MH of older workers, retirement age and working conditions

## Preprocessing EWCS data

Load libraries

In [1]:
import os
import sys

src_path = os.path.abspath("../")
sys.path.append(src_path)

from utils.common import *
from utils.ewcs import *

import_libraries()
import pyreadstat

Load data

In [2]:
ewcs, meta_ewcs = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/EWCS 1991-2015 UKDA ISCO.DTA"
)
df10, meta_df10 = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/UKDA-2010/stata/stata11/ewcs_2010_version_ukda_6_dec_2011.dta"
)

Apply some initial formatting

In [3]:
ewcs = ewcs_preprocessing(ewcs, meta_ewcs)

Calculate social environment index

In [4]:
soc = social_environment_index(ewcs, df10)
ewcs = ewcs.merge(soc, on=["id", "year"], how="left")

JQI social environment
        count       mean        std        min        25%        50%  \
year                                                                   
2010  30836.0  82.983566  14.530474  12.500000  75.000000  87.500000   
2015  32121.0  81.258367  15.335162   4.545455  72.727273  81.818182   

            75%    max  
year                    
2010  93.750000  100.0  
2015  90.909091  100.0  


Calculate prospects index

In [5]:
pro = prospects_index(ewcs, df10)
ewcs = ewcs.merge(pro, on=["id", "year"], how="left")

JQI prospects
        count       mean        std  min   25%   50%   75%    max
year                                                             
2010  38556.0  55.655086  23.024819  0.0  37.5  50.0  75.0  100.0
2015  75016.0  58.081776  24.106244  0.0  50.0  62.5  75.0  100.0


Calculate intensity index (greater index - lower intensity, inverse to original)

In [6]:
int = intensity_index(ewcs, df10)
ewcs = ewcs.merge(int, on=["id", "year"], how="left")

JQI intensity
        count       mean        std       min        25%        50%  \
year                                                                  
2010  27063.0  56.353951  15.846348  4.651163  46.511628  58.139535   
2015  29742.0  46.339840  13.663985  2.325581  37.209302  46.511628   

            75%        max  
year                        
2010  67.441860  95.348837  
2015  55.813953  83.720930  


Add intensity slim original

In [7]:
ewcs = ewcs.rename(columns={"intens_slim": "jqi_intensity_slim"})

Adjust indexes with design weights

In [8]:
indexes = [col for col in ewcs.columns if col.startswith("jqi")]

for index in indexes:
    ewcs[f"{index}_w"] = ewcs[f"{index}"] * ewcs["w4"]

    old_min = ewcs[f"{index}_w"].min()
    old_max = ewcs[f"{index}_w"].max()
    new_min = 0
    new_max = 100

    # ewcs[f"{index}_w"] = (
    #    (ewcs[f"{index}_w"] - old_min) / (old_max - old_min)
    # ) * (new_max - new_min) + new_min

Calculate overall work quality index, normal and weighted

In [9]:
ewcs = sum_wq_index(ewcs)

JQI working quality index (no weights applied)
count    32404.000000
mean        64.073738
std         14.215559
min          0.000000
25%         55.306274
50%         65.647126
75%         74.489796
max        100.000000
Name: jqi_sum, dtype: float64


Aggregate by country and isco

In [10]:
indexes = [col for col in ewcs.columns if col.startswith("jqi")]
ewcs = ewcs[["country", "isco"] + indexes]  # choose columns

# 4 digits
ewcs_4digits = (
    ewcs.groupby(["country", "isco"]).mean().reset_index(drop=False)
)  # aggregate on year-country-isco level
ewcs_4digits = ewcs_4digits.dropna().reset_index(drop=True)  # drop missing values

# 3 digits
ewcs["isco"] = ewcs["isco"] // 10
ewcs_3digits = (
    ewcs.groupby(["country", "isco"]).mean().reset_index(drop=False)
)  # aggregate on year-country-isco level
ewcs_3digits = ewcs_3digits.dropna().reset_index(drop=True)  # drop missing values

Delete isco codes with less than 10 observations

In [11]:
# 4 digits
isco_counts = ewcs_4digits["isco"].value_counts()
values_to_keep = isco_counts[isco_counts >= 10].index
ewcs_4digits = ewcs_4digits[ewcs_4digits["isco"].isin(values_to_keep)]
print(f"4 digit isco codes: {ewcs_4digits.isco.nunique()} out of 436")
# 3 digits
isco_counts = ewcs_3digits["isco"].value_counts()
values_to_keep = isco_counts[isco_counts >= 10].index
ewcs_3digits = ewcs_3digits[ewcs_3digits["isco"].isin(values_to_keep)]
print(f"3 digit isco codes: {ewcs_3digits.isco.nunique()} out of 130")

4 digit isco codes: 301 out of 436
3 digit isco codes: 120 out of 130


In [12]:
301 / 436

0.6903669724770642

In [13]:
120 / 130

0.9230769230769231

Final steps

In [14]:
ewcs_4digits.to_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/results/work_quality_indexes_country_4digits.csv",
    index=False,
)  # save resulting dataset

ewcs_3digits.to_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/results/work_quality_indexes_country_3digits.csv",
    index=False,
)  # save resulting dataset